In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
x_train_full, x_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

input_shape = x_train.shape[1:]

Custom layers

In [3]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [4]:
exponential_layer([-1., 0., 1.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.36787945, 1.        , 2.7182817 ], dtype=float32)>

In [5]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=input_shape),
    keras.layers.Dense(1),
    exponential_layer,
])

model.compile(loss='mse', optimizer='sgd')

model.fit(x_train_scaled, y_train, epochs=5, 
          validation_data=(x_valid_scaled, y_valid))

model.evaluate(x_test_scaled, y_test)

Epoch 1/5
363/363 [==============================] - 0s 735us/step - loss: 0.7242 - val_loss: 0.4193
Epoch 2/5
363/363 [==============================] - 0s 561us/step - loss: 0.4501 - val_loss: 0.3872
Epoch 3/5
363/363 [==============================] - 0s 594us/step - loss: 0.4221 - val_loss: 0.3577
Epoch 4/5
363/363 [==============================] - 0s 591us/step - loss: 0.3905 - val_loss: 0.3630
Epoch 5/5
162/162 [==============================] - 0s 366us/step - loss: 0.3632


0.3632471561431885

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
_________________________________________________________________
lambda (Lambda)              (None, 1)                 0         
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [7]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name='kernel', 
                                      shape=[batch_input_shape[-1], self.units], 
                                      initializer='glorot_normal')
        self.bias = self.add_weight(name='bias', 
                                    shape=[self.units], 
                                    initializer='zeros')
        super().build(batch_input_shape)
        
    def call(self, x):
        return self.activation(x @ self.kernel + self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units': self.units, 
                'activation': keras.activations.serialize(self.activation)}

In [8]:
model = keras.models.Sequential([
    MyDense(30, activation='relu', input_shape=input_shape),
    MyDense(1),
])

model.compile(loss='mse', optimizer='nadam')

model.fit(x_train_scaled, y_train, epochs=2, 
          validation_data=(x_valid_scaled, y_valid))

model.evaluate(x_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 1s 754us/step - loss: 1.8547 - val_loss: 0.7400
Epoch 2/2
162/162 [==============================] - 0s 373us/step - loss: 0.4829


0.4828752279281616

In [9]:
model.save('model_with_custom_layer.h5')

In [10]:
model = keras.models.load_model('model_with_custom_layer.h5', 
                                custom_objects={'MyDense': MyDense})

In [11]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, x):
        x1, x2 = x
        return x1 + x2, x1 * x2
    
    def compute_output_shape(self, batch_input_shape):
        batch_input_shape1, batch_input_shape2 = batch_input_shape
        return [batch_input_shape1, batch_input_shape2]

In [12]:
inputs1 = keras.layers.Input(shape=[2])
inputs2 = keras.layers.Input(shape=[2])
outputs1, outputs2 = MyMultiLayer()((inputs1, inputs2))

In [13]:
def split_data(data):
    columns_count = data.shape[-1]
    half = columns_count // 2
    return data[:, :half], data[:, half:]

In [14]:
x_train_scaled_a, x_train_scaled_b = split_data(x_train_scaled)
x_valid_scaled_a, x_valid_scaled_b = split_data(x_valid_scaled)
x_test_scaled_a, x_test_scaled_b = split_data(x_test_scaled)

x_train_scaled_a.shape, x_train_scaled_b.shape

((11610, 4), (11610, 4))

In [15]:
input_a = keras.layers.Input(shape=x_train_scaled_a.shape[-1])
input_b = keras.layers.Input(shape=x_train_scaled_b.shape[-1])
hidden_a, hidden_b = MyMultiLayer()((input_a, input_b))
hidden_a = keras.layers.Dense(30, activation='selu')(hidden_a)
hidden_b = keras.layers.Dense(30, activation='selu')(hidden_b)
concat = keras.layers.Concatenate()((hidden_a, hidden_b))
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_a, input_b], outputs=[output])

In [16]:
model.compile(loss='mse', optimizer='nadam')

model.fit((x_train_scaled_a, x_train_scaled_b), y_train, epochs=2, 
          validation_data=((x_valid_scaled_a, x_valid_scaled_b), y_valid))

model.evaluate((x_test_scaled_a, x_test_scaled_b), y_test)

Epoch 1/2
363/363 [==============================] - 1s 840us/step - loss: 1.7525 - val_loss: 3.2921
Epoch 2/2
162/162 [==============================] - 0s 404us/step - loss: 1.0700


1.069963812828064

In [17]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
        
    def call(self, x, training=None):
        if training:
            noise = tf.random.normal(tf.shape(x), stddev=self.stddev)
            return x + noise
        else:
            return x
        
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

In [18]:
model = keras.models.Sequential([
    AddGaussianNoise(stddev=1.0),
    keras.layers.Dense(30, activation='selu'),
    keras.layers.Dense(1),
])

In [19]:
model.compile(loss='mse', optimizer='nadam')

model.fit(x_train_scaled, y_train, epochs=2, 
          validation_data=(x_valid_scaled, y_valid))

model.evaluate(x_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 1s 732us/step - loss: 2.2520 - val_loss: 1.7259
Epoch 2/2
162/162 [==============================] - 0s 379us/step - loss: 0.7768


0.7768131494522095